In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [4]:
result = pd.read_csv("results_dyna_5_old.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","total_demand",
                                            "serviced_demand","stay_demand","trip_rate","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

count = 0
result_col = ['profit','ave_pax','ave_dist','ave_time',
              'ave_time_offer','ave_time_offer_s','ave_time_pickup',
              'ave_link_time','trip_rate']
compare_stra = np.zeros((3,len(result_col)))
for congest in [0.8,1.0,1.2]:
    for demand in [1,2,4]:
        for supply in [0.75,1.0,1.25,1.5]:
            for prate in [0.4,0.6,0.8]:
                for tax_c in [0.0,-0.025]:
                    for tax_d in [0.0,-0.25]:
                        count += 1
                        temp = result.loc[(result['congestion'] == congest) & (result['demand'] == demand) & \
                                          (result['supply'] == supply) & (result['prate'] == prate) & \
                                          (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                        pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])
                        full_temp = np.asarray(temp.loc[(result['strategy'] == "Full"),result_col])
                        pricing_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing"),result_col])
                        assort_temp = np.asarray(temp.loc[(result['strategy'] == "Assort"),result_col])
                        compare_stra[0,:] += ((pricing_opt_temp - full_temp) / pricing_opt_temp)[0,:]
                        compare_stra[1,:] += ((pricing_opt_temp - assort_temp) / pricing_opt_temp)[0,:]
                        compare_stra[2,:] += ((pricing_opt_temp - pricing_temp) / pricing_opt_temp)[0,:]
            
compare_stra = - compare_stra / count * 100
compare_stra

array([[ -1.77878790e+01,   1.03803565e+01,  -1.71748501e-01,
          8.97668558e+00,   5.02929937e+02,   7.13600584e+02,
          1.51047444e+01,   3.55399243e+00,   3.33086672e-02],
       [ -1.62934046e+01,  -4.40537038e+00,  -7.94001868e+00,
          3.36696833e-01,   4.65304769e+02,   7.11951907e+02,
          1.48928357e+01,   4.26499145e+00,  -1.99080955e+00],
       [ -1.64779914e+01,   6.53639983e+00,   2.44914920e-01,
          8.32878051e+00,   6.61401876e+02,   9.68382179e+02,
          1.64635868e+01,   3.99305073e+00,  -3.22101190e-01]])

In [14]:
result = pd.read_csv("results_dyna_5.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","total_demand",
                                            "serviced_demand","stay_demand","trip_rate","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

del result['public'],result['private']
del result['revenue'],result['cost'],result['total_pax'],result['ave_pax'],result['ave_dist'],result['ave_time']
del result['ave_time_offer'],result['ave_time_offer_s'],result['ave_time_pickup']
result['shared_rate'] = result['shared_pax'] / result['serviced_demand']
result['total_shared_rate'] = result['shared_pax'] / (result['serviced_demand'] + result['stay_demand'])

result_col = ["ave_link_time","trip_rate"]
background = np.array([[1.67533,0.744199],
                       [1.64807,0.743382],
                       [1.85175,0.742263],
                       [2.29136,0.724696],
                       [2.22969,0.72743],
                       [2.612,0.718507],
                       [3.09868,0.700391],
                       [3.0063,0.701303],
                       [3.57836,0.69169]
                      ])
compare_stra = np.zeros((9,6))
row_id = -1
count = 0
for congest in [0.8,1.0,1.2]:
    for demand in [1,2,4]:
        row_id += 1
        background_temp = background[row_id,:]
        for supply in [0.75,1.0,1.25,1.5]:
            for prate in [0.4,0.6,0.8]:
                for tax_c in [0.0,-0.025]:
                    for tax_d in [0.0,-0.25]:
                        count += 1
                        temp = result.loc[(result['congestion'] == congest) & (result['demand'] == demand) & \
                                          (result['supply'] == supply) & (result['prate'] == prate) & \
                                          (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                        pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])[0,:]
                        full_temp = np.asarray(temp.loc[(result['strategy'] == "Full"),result_col])[0,:]
                        shared_temp = np.asarray(temp.loc[(result['strategy'] == "Pool"),result_col])[0,:]
                        compare_stra[row_id,[2,5]] += ((pricing_opt_temp - background_temp) / background_temp)
                        compare_stra[row_id,[0,3]] += ((full_temp - background_temp) / background_temp)
                        compare_stra[row_id,[1,4]] += ((shared_temp - background_temp) / background_temp)
            
compare_stra = compare_stra / count * 100 * 9
compare_stra

array([[  2.03500952,   1.56700471,   1.78657588,  19.19443198,
         18.09747796,  17.60216353],
       [  5.29094891,   4.02182553,   4.60289662,  18.90216493,
         17.95129388,  17.35781817],
       [ 48.89848567,  31.77946537,  40.03051168,  14.67461949,
         15.25021848,  14.29274675],
       [  2.62711519,   2.0510381 ,   2.44564545,  17.85235637,
         17.22237957,  16.8806012 ],
       [  7.16636431,   5.4351726 ,   6.61905355,  16.64894732,
         16.32325161,  15.7187089 ],
       [ 65.93740429,  44.94543613,  58.43423781,   9.94156111,
         12.2693742 ,  10.27869828],
       [  3.91500709,   3.08535274,   3.06172709,  15.12515271,
         15.23415433,  14.88220508],
       [ 12.32456979,   9.15629262,   8.12773731,  13.37301483,
         14.09134734,  13.96167206],
       [ 71.76539439,  54.49702634,  47.86820406,   4.31590862,
          7.4028346 ,   8.3611848 ]])

In [69]:
result = pd.read_csv("results_dyna_2.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","total_demand",
                                            "serviced_demand","stay_demand","trip_rate","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

del result['public'],result['private']
#del result['revenue'],result['cost'],result['total_pax'],result['ave_pax'],result['ave_dist'],result['ave_time']
del result['ave_time_offer'],result['ave_time_offer_s'],result['ave_time_pickup']
result['shared_rate'] = result['shared_pax'] / result['serviced_demand']
result['total_shared_rate'] = result['shared_pax'] / (result['serviced_demand'] + result['stay_demand'])

result_col = ["ave_link_time","trip_rate"]
background = np.array([[1.66151,0.844788],
                       [1.61626,0.845357],
                       [1.63496,0.846372],
                       [2.27144,0.832714],
                       [2.17762,0.833439],
                       [2.31931,0.831695],
                       [3.06912,0.815228],
                       [2.93096,0.816399],
                       [3.67519,0.811018]
                      ])
compare_stra = np.zeros((9,6))
row_id = -1
count = 0
for congest in [0.8,1.0,1.2]:
    for demand in [1,2,4]:
        row_id += 1
        background_temp = background[row_id,:]
        for supply in [0.5,0.75,1.0,1.25]:
            for prate in [0.4,0.6,0.8]:
                for tax_c in [0.0,-0.025]:
                    for tax_d in [0.0,-0.25]:
                        count += 1
                        temp = result.loc[(result['congestion'] == congest) & (result['demand'] == demand) & \
                                          (result['supply'] == supply) & (result['prate'] == prate) & \
                                          (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                        pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])[0,:]
                        full_temp = np.asarray(temp.loc[(result['strategy'] == "Full"),result_col])[0,:]
                        shared_temp = np.asarray(temp.loc[(result['strategy'] == "Pool"),result_col])[0,:]
                        compare_stra[row_id,[2,5]] += ((pricing_opt_temp - background_temp) / background_temp)
                        compare_stra[row_id,[0,3]] += ((full_temp - background_temp) / background_temp)
                        compare_stra[row_id,[1,4]] += ((shared_temp - background_temp) / background_temp)
            
compare_stra = compare_stra / count * 100 * 9
compare_stra

array([[  0.71904974,   0.4521991 ,   0.5702644 ,   7.91971378,
          7.29609579,   7.25770598],
       [  1.64886838,   0.98849608,   1.29711752,   7.84846422,
          7.25263853,   7.21327893],
       [  5.94508683,   3.19568216,   5.48017994,   7.44025431,
          6.98041415,   6.86276001],
       [  1.04572063,   0.69464011,   0.83168607,   7.22447923,
          6.78819999,   6.68719242],
       [  2.63674715,   1.73677761,   2.19892206,   7.15757242,
          6.7720488 ,   6.68999171],
       [ 16.54616567,   8.05670652,  12.18640098,   6.35418733,
          6.44787152,   6.24868241],
       [  1.78421801,   1.25971913,   1.35012235,   6.29053569,
          6.11363477,   6.14454637],
       [  4.82882173,   3.31277266,   3.5846759 ,   6.06104776,
          6.02794763,   5.8867585 ],
       [ 36.68358529,  18.48322241,  15.59698274,   2.78730969,
          4.22251726,   4.33508412]])

In [70]:
result_s = result.loc[(result['congestion'] == 1.2) & (result['demand'] == 2),:]
background_s = background[7,:]
result_s['ave_link_time_pc'] = result_s['ave_link_time']/background_s[0] - 1
result_s['trip_rate_pc'] = result_s['trip_rate']/background_s[1] - 1

def get_re_table_2(re,name,stra):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.5,0.75,1.0,1.25]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_c in [0.0,-0.025]:
                for tax_d in [0.0,-0.25]:
                    temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                                  (re['tax_cong'] == tax_c) & (re['tax_demand'] == tax_d) & \
                                  (re['strategy'] == stra),name]
                    re_out[row_id,col_id] += np.asarray(temp)[0]
    re_out = re_out / 4 * 100
    return re_out

result_col = ["ave_link_time_pc","trip_rate_pc","shared_rate","total_shared_rate","ave_dist","stay_demand"]
result_name = result_col[0]
print get_re_table_2(result_s,result_name,"Pricing Opt"),'\n\n',\
    get_re_table_2(result_s,result_name,"Pool"),'\n\n',\
    get_re_table_2(result_s,result_name,"Full")

[[ 2.1376102   3.20159606  4.75108838  4.8837241 ]
 [ 1.82329339  2.73135764  4.12544013  4.97482054]
 [ 2.47222753  2.86288452  4.03630551  5.01576275]] 

[[ 1.98510044  2.9183271   4.03809673  4.51072345]
 [ 2.14503098  3.23827347  3.88703701  4.19444482]
 [ 2.35008325  3.25021495  3.51045391  3.72548585]] 

[[ 2.45823894  3.73171248  5.21987676  6.60210307]
 [ 2.74346972  4.25867293  5.76952603  6.78148115]
 [ 3.08951675  4.71927287  6.12768513  6.44430494]]


In [117]:
result = pd.read_csv("results_dyna_5.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","total_demand",
                                            "serviced_demand","stay_demand","trip_rate","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

del result['public'],result['private']
#del result['revenue'],result['cost'],result['total_pax'],result['ave_pax'],result['ave_dist'],result['ave_time']
del result['ave_time_offer'],result['ave_time_offer_s'],result['ave_time_pickup']
result['shared_rate'] = result['shared_pax'] / result['serviced_demand']
result['total_shared_rate'] = result['shared_pax'] / (result['serviced_demand'] + result['stay_demand'])
result['ave_pax_dist'] = result['ave_dist'] / result['total_pax']

result_col = ["ave_link_time","trip_rate"]
background = np.array([[1.67533,0.744199],
                       [1.64807,0.743382],
                       [1.85175,0.742263],
                       [2.29136,0.724696],
                       [2.22969,0.72743],
                       [2.612,0.718507],
                       [3.09868,0.700391],
                       [3.0063,0.701303],
                       [3.57836,0.69169]
                      ])
compare_stra = np.zeros((9,6))
row_id = -1
count = 0
for congest in [0.8,1.0,1.2]:
    for demand in [1,2,4]:
        row_id += 1
        background_temp = background[row_id,:]
        for supply in [0.75,1.0,1.25,1.5]:
            for prate in [0.4,0.6,0.8]:
                for tax_c in [0.0,-0.025]:
                    for tax_d in [0.0,-0.25]:
                        count += 1
                        temp = result.loc[(result['congestion'] == congest) & (result['demand'] == demand) & \
                                          (result['supply'] == supply) & (result['prate'] == prate) & \
                                          (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                        pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])[0,:]
                        full_temp = np.asarray(temp.loc[(result['strategy'] == "Full"),result_col])[0,:]
                        shared_temp = np.asarray(temp.loc[(result['strategy'] == "Pool"),result_col])[0,:]
                        compare_stra[row_id,[2,5]] += ((pricing_opt_temp - background_temp) / background_temp)
                        compare_stra[row_id,[0,3]] += ((full_temp - background_temp) / background_temp)
                        compare_stra[row_id,[1,4]] += ((shared_temp - background_temp) / background_temp)
            
compare_stra = compare_stra / count * 100 * 9
compare_stra

array([[  2.03500952,   1.56700471,   1.78657588,  19.19443198,
         18.09747796,  17.60216353],
       [  5.29094891,   4.02182553,   4.60289662,  18.90216493,
         17.95129388,  17.35781817],
       [ 48.89848567,  31.77946537,  40.03051168,  14.67461949,
         15.25021848,  14.29274675],
       [  2.62711519,   2.0510381 ,   2.44564545,  17.85235637,
         17.22237957,  16.8806012 ],
       [  7.16636431,   5.4351726 ,   6.61905355,  16.64894732,
         16.32325161,  15.7187089 ],
       [ 65.93740429,  44.94543613,  58.43423781,   9.94156111,
         12.2693742 ,  10.27869828],
       [  3.91500709,   3.08535274,   3.06172709,  15.12515271,
         15.23415433,  14.88220508],
       [ 12.32456979,   9.15629262,   8.12773731,  13.37301483,
         14.09134734,  13.96167206],
       [ 71.76539439,  54.49702634,  47.86820406,   4.31590862,
          7.4028346 ,   8.3611848 ]])

In [123]:
result_s = result.loc[(result['congestion'] == 1.2) & (result['demand'] == 4),:]
background_s = background[8,:]
result_s['ave_link_time_pc'] = result_s['ave_link_time']/background_s[0] - 1
result_s['trip_rate_pc'] = result_s['trip_rate']/background_s[1] - 1

def get_re_table_5(re,name,stra):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.75,1.0,1.25,1.5]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_c in [0.0,-0.025]:
                for tax_d in [0.0,-0.25]:
                    temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                                  (re['tax_cong'] == tax_c) & (re['tax_demand'] == tax_d) & \
                                  (re['strategy'] == stra),name]
                    re_out[row_id,col_id] += np.asarray(temp)[0]
    re_out = re_out / 4
    return re_out

def get_tax_c_eff_5(re,name):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.75,1.0,1.25,1.5]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_d in [0.0,-0.25]:
                temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                              (re['tax_demand'] == tax_d) & \
                              (re['strategy'] == "Pricing Opt"),['tax_cong',name]]
                temp1 = np.asarray(temp.loc[(temp['tax_cong'] == -0.025),name])[0]
                temp2 = np.asarray(temp.loc[(temp['tax_cong'] == 0.0),name])[0]
                re_out[row_id,col_id] += (float(temp1)/float(temp2) - 1)
    re_out = re_out / 2 * 100
    return re_out

def get_tax_d_eff_5(re,name):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.75,1.0,1.25,1.5]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_c in [0.0,-0.025]:
                temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                              (re['tax_cong'] == tax_c) & \
                              (re['strategy'] == "Pricing Opt"),['tax_demand',name]]
                temp1 = np.asarray(temp.loc[(temp['tax_demand'] == -0.25),name])[0]
                temp2 = np.asarray(temp.loc[(temp['tax_demand'] == 0.0),name])[0]
                re_out[row_id,col_id] += (float(temp1)/float(temp2) - 1)
    re_out = re_out / 2 * 100
    return re_out

result_col = ["ave_link_time_pc","trip_rate_pc","total_shared_rate",
              "ave_link_time","trip_rate","ave_pax_dist","ave_time","stay_demand"]

result_name = result_col[6]
print get_re_table_5(result_s,result_name,"Pricing Opt"),'\n\n',\
    get_re_table_5(result_s,result_name,"Pool"),'\n\n',\
    get_re_table_5(result_s,result_name,"Full")

#result_name = result_col[4]
#print get_tax_c_eff_5(result_s,result_name),'\n\n',get_tax_d_eff_5(result_s,result_name)

[[ 790.39375  733.87     718.88425  680.17175]
 [ 803.34975  768.02275  706.58075  690.6765 ]
 [ 844.354    748.735    692.56025  655.12075]] 

[[ 867.44675  830.2395   796.4545   763.503  ]
 [ 848.8065   806.26225  765.66825  722.47825]
 [ 823.35825  770.57125  711.1335   633.38   ]] 

[[ 887.27875  847.2815   816.51725  777.7915 ]
 [ 875.72975  836.20725  802.6195   762.257  ]
 [ 864.0925   824.37575  786.72875  746.53275]]


In [143]:
result = pd.read_csv("results_dyna_21.csv",
                     names = ["strategy","congestion","demand","supply","prate","tax_cong","tax_demand",
                                            "private","revenue","cost","tax","profit","total_pax",
                                            "single_pax","shared_pax","ave_pax","ave_dist","ave_time",
                                            "public","ave_link_time","ave_density","total_demand",
                                            "serviced_demand","stay_demand","trip_rate","ave_single_time",
                                            "ave_shared_time","ave_stay_time","ave_exit_time","ave_time_offer",
                                            "ave_time_offer_s","ave_time_pickup"])

del result['public'],result['private']
del result['revenue'],result['cost'],result['total_pax'],result['ave_pax']
del result['ave_time_offer'],result['ave_time_offer_s'],result['ave_time_pickup']
result['shared_rate'] = result['shared_pax'] / result['serviced_demand']
result['total_shared_rate'] = result['shared_pax'] / (result['serviced_demand'] + result['stay_demand'])

result_col = ["ave_link_time","trip_rate"]
background = np.array([[1.66151,0.844788],
                       [1.61626,0.845357],
                       [1.63496,0.846372],
                       [2.27144,0.832714],
                       [2.17762,0.833439],
                       [2.31931,0.831695],
                       [3.06912,0.815228],
                       [2.93096,0.816399],
                       [3.67519,0.811018]
                      ])
compare_stra = np.zeros((9,4))
row_id = -1
count = 0
for congest in [0.8,1.0,1.2]:
    for demand in [1,2,4]:
        row_id += 1
        background_temp = background[row_id,:]
        for supply in [0.5,0.75,1.0,1.25]:
            for prate in [0.4,0.6,0.8]:
                for tax_c in [0.0,-0.025]:
                    for tax_d in [0.0,-0.25]:
                        count += 1
                        temp = result.loc[(result['congestion'] == congest) & (result['demand'] == demand) & \
                                          (result['supply'] == supply) & (result['prate'] == prate) & \
                                          (result['tax_cong'] == tax_c) & (result['tax_demand'] == tax_d),:]
                        pricing_opt_temp = np.asarray(temp.loc[(result['strategy'] == "Pricing Opt"),result_col])[0,:]
                        shared_temp = np.asarray(temp.loc[(result['strategy'] == "Pool"),result_col])[0,:]
                        compare_stra[row_id,[1,3]] += ((pricing_opt_temp - background_temp) / background_temp)
                        compare_stra[row_id,[0,2]] += ((shared_temp - background_temp) / background_temp)
            
compare_stra = compare_stra / count * 100 * 9
compare_stra

array([[ 0.22183636,  0.30726317,  8.19748564,  8.06086113],
       [ 0.29473909,  0.49856613,  8.17171522,  8.04411627],
       [-0.90313117,  0.10108555,  8.02818776,  7.87858107],
       [ 0.41778747,  0.54833057,  7.80218358,  7.54970344],
       [ 0.90191164,  1.30958148,  7.8170388 ,  7.59800157],
       [-2.19926221,  1.02701163,  7.90794702,  7.64738225],
       [ 1.03256389,  1.11204515,  7.08453136,  6.95390584],
       [ 2.44218902,  2.75142359,  7.0606458 ,  6.73646046],
       [ 2.80207735,  0.84535312,  6.31169972,  6.21615365]])

In [149]:
result_s = result.loc[(result['congestion'] == 1.2) & (result['demand'] == 4),:]
background_s = background[8,:]
result_s['ave_link_time_pc'] = result_s['ave_link_time']/background_s[0] - 1
result_s['trip_rate_pc'] = result_s['trip_rate']/background_s[1] - 1

def get_re_table_2(re,name,stra):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.5,0.75,1.0,1.25]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_c in [0.0,-0.025]:
                for tax_d in [0.0,-0.25]:
                    temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                                  (re['tax_cong'] == tax_c) & (re['tax_demand'] == tax_d) & \
                                  (re['strategy'] == stra),name]
                    re_out[row_id,col_id] += np.asarray(temp)[0]
    re_out = re_out / 4
    return re_out

def get_tax_c_eff_2(re,name):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.5,0.75,1.0,1.25]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_d in [0.0,-0.25]:
                temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                              (re['tax_demand'] == tax_d) & \
                              (re['strategy'] == "Pricing Opt"),['tax_cong',name]]
                temp1 = np.asarray(temp.loc[(temp['tax_cong'] == -0.025),name])[0]
                temp2 = np.asarray(temp.loc[(temp['tax_cong'] == 0.0),name])[0]
                re_out[row_id,col_id] += (float(temp1)/float(temp2) - 1)
    re_out = re_out / 2 * 100
    return re_out

def get_tax_d_eff_2(re,name):
    re_out = np.zeros((3,4))
    col_id = -1
    for supply in [0.5,0.75,1.0,1.25]:
        col_id += 1
        row_id = -1
        for prate in [0.4,0.6,0.8]:
            row_id += 1
            for tax_c in [0.0,-0.025]:
                temp = re.loc[(re['supply'] == supply) & (re['prate'] == prate) & \
                              (re['tax_cong'] == tax_c) & \
                              (re['strategy'] == "Pricing Opt"),['tax_demand',name]]
                temp1 = np.asarray(temp.loc[(temp['tax_demand'] == -0.25),name])[0]
                temp2 = np.asarray(temp.loc[(temp['tax_demand'] == 0.0),name])[0]
                re_out[row_id,col_id] += (float(temp1)/float(temp2) - 1)
    re_out = re_out / 2 * 100
    return re_out

result_col = ["ave_link_time_pc","trip_rate_pc","total_shared_rate",
              "ave_link_time","trip_rate","ave_dist","ave_time",
              "ave_single_time","ave_shared_time","ave_stay_time","ave_exit_time"]

result_name = result_col[5]
print get_re_table_2(result_s,result_name,"Pricing Opt"),'\n\n',\
    get_re_table_2(result_s,result_name,"Pool")

#result_name = result_col[4]
#print get_tax_c_eff_2(result_s,result_name),'\n\n',get_tax_d_eff_2(result_s,result_name)

[[ 329.9995   274.2025   242.91225  217.2085 ]
 [ 315.93975  280.46225  240.5895   195.364  ]
 [ 316.5605   272.406    232.28275  202.29675]] 

[[ 294.85525  261.27825  237.3075   215.51375]
 [ 278.4435   245.31825  221.85025  188.51675]
 [ 261.101    229.53275  191.30625  158.96975]]
